### A. Import relevant dependencies

In [1]:
import os
import pickle
import sys

import numpy as np
import torch
import yaml
import matplotlib.pyplot as plt
from torch_geometric.loader import NeighborLoader
from torch_geometric.utils import to_undirected
from torch_geometric.utils.convert import from_networkx
from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR

try:
    # This will work in scripts where __file__ is defined
    current_dir = os.path.dirname(os.path.abspath(__file__))
    # Assuming "src" is parallel to the script folder
    project_root = os.path.abspath(os.path.join(current_dir, ".."))
except NameError:
    # In notebooks __file__ is not defined: assume we're in notebooks/
    project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from main.graph_formatting_utils import prepare_graph_for_gnn
from main.loss_functions import unsupervised_loss_V0, unsupervised_loss_V1
from main.node_embedding_models import GraphSAGE, GraphSAGE_V2
from main.training_utils import train_in_cpu, train_in_gpu

### B. Check CUDA status

In [2]:
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")
print("CUDA Device Count:", torch.cuda.device_count())

CUDA Available: True
GPU Name: NVIDIA L40S
CUDA Device Count: 1


### C. Load config data

In [3]:
# Load GraphSAGE config file
config_file_path = os.path.join(
    project_root, "config", "graphsage_config.yaml"
    )

with open(config_file_path, "r") as f:
    config = yaml.safe_load(f)

print(config)

input_graph_file_name = config['input_data']['graph_file_name']
input_graph_embedding_dim = config['model_params']['channels'][0] 

graphsage_channels = config['model_params']['channels']

loader_num_neighbors = config['loader_params']['num_neighbors']
loader_batch_size = config['loader_params']['batch_size']
loader_shuffle = config['loader_params']['shuffle']

training_num_epochs = config['training_params']['num_epochs']
optimizer_learning_rate = config['optimizer_params']['learning_rate']

bundle_tag = config["bundle_tag"]

{'model_params': {'channels': [1024, 768, 512]}, 'loader_params': {'num_neighbors': [15, 15], 'batch_size': 256, 'shuffle': True}, 'input_data': {'graph_file_name': 'RIZIV_hackathon_main/RIZIV_base_hybrid_graph_wsem.pkl'}, 'optimizer_params': {'learning_rate': 0.0005}, 'training_params': {'num_epochs': 10}, 'bundle_tag': 'RIZIV_test'}


### D. Load data and train embedding model

In [ ]:
# Load and format graph
graph_path = os.path.join(project_root, "data", input_graph_file_name)
with open(graph_path, "rb") as f:
    G = pickle.load(f)

formatted_G, incidences = prepare_graph_for_gnn(G, embedding_dim=input_graph_embedding_dim)

# Convert the NetworkX graph to a PyTorch Geometric Data object
data = from_networkx(formatted_G)

# Ensure the graph is undirected
data.edge_index = to_undirected(data.edge_index)

# Create data attribute "x" containing the embeddings of each node complying with the PyTorch Geometric API
data.x = data.embedding
del data.embedding

# Instantiate the GraphSAGE model
model = GraphSAGE_V2(
    channels=graphsage_channels
)

# Set device for model training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model = model.to(device)

# Instantiate the NeighborLoader for mini-batch training
train_loader = NeighborLoader(
    data,
    num_neighbors=loader_num_neighbors,  #  neighbors for the first layer, 15 for the second
    batch_size=loader_batch_size,  # Batch size
    shuffle=loader_shuffle
)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_learning_rate) # (old optimizer)

# # 1) Instantiate the optimizer with weight decay regularization
# optimizer = torch.optim.Adam(
#     model.parameters(),
#     lr=optimizer_learning_rate,  # e.g. 5e-4
#     weight_decay=1e-6            # L2 regularization coefficient
# )
# 
# # 2) Compute total number of optimization steps and warm‑up steps
# num_epochs   = training_num_epochs         # from your config
# steps_per_ep = len(train_loader)           # number of batches per epoch
# total_steps  = num_epochs * steps_per_ep
# warmup_steps = int(0.05 * total_steps)      # e.g. 10% of all training steps
# 
# # 3) Define two schedulers: a linear warm‑up, then cosine decay
# warmup_scheduler = LinearLR(
#     optimizer,
#     start_factor=0.1,    # start at 0% of target LR
#     end_factor=1.0,      # end at 100% of target LR
#     total_iters=warmup_steps
# )
# 
# decay_scheduler = CosineAnnealingLR(
#     optimizer,
#     T_max=total_steps - warmup_steps,  # remaining steps after warm‑up
#     eta_min=1e-6                       # minimum LR at end of decay
# )

# Define scaler if GPU is available
scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

if not torch.cuda.is_available():
    training_outputs = train_in_cpu(model, train_loader, optimizer, num_epochs=training_num_epochs, loss_fn=unsupervised_loss_V1, debug=False, plot_eval=True)

elif torch.cuda.is_available():
    training_outputs = train_in_gpu(model, train_loader, optimizer, num_epochs=training_num_epochs, loss_fn=unsupervised_loss_V1, debug=False, plot_eval=True)


/home/diomedea/.pyenv/versions/3.11.6/envs/graphsage/lib/python3.11/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
Training epochs:   0%|          | 0/10 [00:00<?, ?it/s]

### E. Save embeddings, models weights and config

In [ ]:
#  Add Graph_SAGE embeddings to the baseline graph

## -> Move data to the same device as the model
device = next(model.parameters()).device
data_x = data.x.to(device)
data_edge_index = data.edge_index.to(device)

## -> Obtain final embeddings from the trained model
with torch.no_grad():
    final_emb = model(data_x, data_edge_index)  # shape [num_nodes, embedding_dim]
    final_emb_np = final_emb.cpu().numpy()

## -> Add them back to the cleaned_G graph
list_of_nodes = list(G.nodes())  # Must match the node ordering in data
for i, node in enumerate(list_of_nodes):
    # Store as a NumPy array (or you could store as a list if you prefer)
    G.nodes[node]["hybrid_embedding"] = final_emb_np[i]

# Define tags for the output files
training_n_hops = len(graphsage_channels)-1
channels_str = "-".join([str(i) for i in graphsage_channels])
training_num_epochs

# Check if retrieval_bundles directory exists
retrieval_bundles_dir = os.path.join(project_root, "data", "retrieval_bundles")
os.makedirs(retrieval_bundles_dir, exist_ok=True)

# Create bundle directory
bundle_directory = os.path.join(
    retrieval_bundles_dir, f"{bundle_tag}_{training_n_hops}hop_{training_num_epochs}epochs_{channels_str}"
)

os.makedirs(bundle_directory, exist_ok=True)

# Save the processed graph
output_graph_bundle_path = os.path.join(bundle_directory, "graph.pkl")
with open(output_graph_bundle_path, "wb") as f:
    pickle.dump(G, f)

# Save the trained model
output_model_bundle_path = os.path.join(bundle_directory, "graphsage.pth")
torch.save(model.state_dict(), output_model_bundle_path)

# Save config dictionary
config_bundle_path = os.path.join(bundle_directory, "config.yaml")
with open(config_bundle_path, "w") as f:
    yaml.dump(config, f)

# Extract figures from dictionary
loss_fig = training_outputs.pop('loss_fig')
norm_fig = training_outputs.pop('norm_fig')

# Save figures
loss_fig.savefig(os.path.join(bundle_directory, 'loss_evolution.jpg'))
norm_fig.savefig(os.path.join(bundle_directory, 'norm_evolution.jpg'))
plt.close('all')

# Save metrics dictionary (now without figures)
metrics_path = os.path.join(bundle_directory, "training_metrics.pkl")
with open(metrics_path, "wb") as f:
    pickle.dump(training_outputs, f)

In [ ]:
#def get_new_node_embedding(model, new_feature, device, self_loop=True):
#    """
#    Generate an embedding for a new node using the trained GraphSAGE model.
#    
#    Parameters:
#      model (torch.nn.Module): The trained GraphSAGE model.
#      new_feature (torch.Tensor): The new node's initial features of shape (1024,).
#      device (torch.device): The device (e.g., cuda) on which the model is located.
#      self_loop (bool, default=True): If True, adds a self-loop edge (node connected to itself).
#                                      This simulates neighbor aggregation when no neighbors exist.
#    
#    Returns:
#      torch.Tensor: The new node's embedding of shape (out_channels,).
#    """
#    # Set the model to evaluation mode.
#    model.eval()
#    
#    # Move the new node's features to the correct device.
#    new_feature = new_feature.to(device)
#    
#    if self_loop:
#        # Create a self-loop edge_index. This indicates that the node is connected to itself.
#        # The edge_index tensor must have shape [2, num_edges]; here we create a single edge (0,0).
#        edge_index = torch.tensor([[0], [0]], dtype=torch.long, device=device)
#    else:
#        # Alternatively, if you prefer no edges, you can pass an empty edge_index.
#        # Note: Without a self-loop, the model may not transform the features as intended.
#        edge_index = torch.empty((2, 0), dtype=torch.long, device=device)
#    
#    # GraphSAGE expects a batch dimension, so unsqueeze new_feature to shape [1, 1024].
#    with torch.no_grad():
#        new_embedding = model(new_feature.unsqueeze(0), edge_index)
#    
#    # Remove the batch dimension to return a tensor of shape [out_channels].
#    return new_embedding.squeeze(0)
#
## Example usage:
## Assuming 'model' is your trained GraphSAGE and you have a new node feature vector of size 1024.
#new_feature = torch.randn(1024)  # Replace with the actual feature vector for the new node.
#device = next(model.parameters()).device  # Get the device from the model.
#new_node_embedding = get_new_node_embedding(model, new_feature, device, self_loop=True)
#print("New node embedding shape:", new_node_embedding.shape)
#